In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import norm,skew
from statsmodels.graphics.gofplots import qqplot

In [ ]:
data = pd.read_csv('/kaggle/input/zomato-bangalore-restaurants/zomato.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
data.drop(['url','address','phone'],axis=1,inplace=True)

In [ ]:
data.describe(include='all')

In [ ]:
data[['rate','outof']] = data['rate'].str.split('/',expand=True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.drop('outof',axis=1,inplace=True)

In [ ]:
data.dtypes

In [ ]:
data['rate'].unique()

In [ ]:
data.drop(data[data['rate'] == '-'].index,inplace=True)

In [ ]:
data.shape

In [ ]:
data[data['rate'] == 'NEW']

In [ ]:
data[data['rate'] == 'NEW'].shape

In [ ]:
data['rate'] = data['rate'].replace('NEW',np.nan)

In [ ]:
data[data['votes'] == 0].shape

In [ ]:
data['rate'].astype('float')

In [ ]:
sns.distplot(data['rate'],fit=norm)

In [ ]:
#from sklearn.impute import KNNImputer

In [ ]:
#imputer = KNNImputer(n_neighbors=3)

In [ ]:
data['location'].unique()

In [ ]:
plt.figure(figsize=(20,20))
sns.countplot(y='location',data=data,palette = "Set1",order=data['location'].value_counts().iloc[:50].index)

In [ ]:
plt.figure(figsize=(20,20))
sns.countplot(y='rest_type',data=data,palette = "Set1",order=data['rest_type'].value_counts().iloc[:30].index)

In [ ]:
plt.figure(figsize=(20,20))
sns.countplot(y='dish_liked',data=data,palette = "Set1",order=data['dish_liked'].value_counts().iloc[:50].index)

In [ ]:
plt.figure(figsize=(20,20))
sns.countplot(y='cuisines',data=data,palette = "Set1",order=data['cuisines'].value_counts().iloc[:60].index)

In [ ]:
data.head()

In [ ]:
data['approx_cost(for two people)'].dtypes

In [ ]:
data['approx_cost(for two people)']

In [ ]:
data.rename(columns={'approx_cost(for two people)': 'average_cost', 'listed_in(city)': 'locality','listed_in(type)': 'restaurant_type'}, inplace=True)

In [ ]:
data['average_cost'].unique()

In [ ]:
data['average_cost'] = data['average_cost'].astype(str).apply(lambda x: x.replace(',',''))
data['average_cost'] = data['average_cost'].astype('float')
fig, ax = plt.subplots(figsize=[16,4])
sns.distplot(data['average_cost'],ax=ax)
ax.set_title('average_cost')

In [ ]:
data['online_order'] = data['online_order'].map({'Yes':1,'No':0})

In [ ]:
online_orders=pd.crosstab(data['rate'],data['online_order'])

In [ ]:
online_orders

In [ ]:
plt.figure(figsize=(20,15))
online_orders.plot.bar(stacked=True)
plt.legend(title='Online orders V/S rating')

In [ ]:
book_table=pd.crosstab(data['rate'],data['book_table'])
plt.figure(figsize=(20,15))
book_table.plot.bar(stacked=True)
plt.legend(title='Book table V/S rating')

In [ ]:
data['location']

In [ ]:
data['rate'].dtypes

In [ ]:
data['rate'] = data['rate'].astype('float')

In [ ]:
location = data.groupby(['location'])['rate'].mean().reset_index()
location.head(50).sort_values('rate', ascending=False).style.background_gradient(cmap='Greens')

In [ ]:
plt.figure(figsize=(15,20))
sns.barplot(x='location',y='rate',data=location[:50])
plt.xticks(rotation=90)

In [ ]:
restrauent_type = data.groupby(['restaurant_type'])['rate'].mean().reset_index()
restrauent_type.head(50).sort_values('rate', ascending=False).style.background_gradient(cmap='Greens')

In [ ]:
location_with_price = data.groupby(['location'])['average_cost'].mean().reset_index()
location_with_price.head(50).sort_values('average_cost', ascending=False).style.background_gradient(cmap='Greens')

In [ ]:
plt.figure(figsize=(15,20))
sns.barplot(x='location',y='average_cost',data=location_with_price[:50])
plt.xticks(rotation=90)

In [ ]:
rest_type_with_price = data.groupby(['rest_type'])['average_cost'].mean().reset_index()
rest_type_with_price.head(50).sort_values('average_cost', ascending=False).style.background_gradient(cmap='Greens')

In [ ]:
plt.figure(figsize=(15,20))
sns.barplot(x='rest_type',y='average_cost',data=rest_type_with_price[:50])
plt.xticks(rotation=90)

In [ ]:
data.head()

In [ ]:
restrauent_type_with_price = data.groupby(['restaurant_type'])['average_cost'].mean().reset_index()
restrauent_type_with_price.head(50).sort_values('average_cost', ascending=False).style.background_gradient(cmap='Greens')

In [ ]:
X= data.drop_duplicates(subset='name',keep='first')

In [ ]:
X.shape

In [ ]:
type(X)

In [ ]:
newdf=X[['name','average_cost','locality','rest_type','cuisines','rate','restaurant_type','online_order','book_table','dish_liked','cuisines']].groupby(['average_cost'], sort = True)
#newdf=newdf.sort_values(by=['average_cost'])

In [ ]:
X= data.drop_duplicates(subset='name',keep='first')
# dups_name = X1.pivot_table(index=['name'],aggfunc='size')
newdf=X[['name','average_cost','locality','rest_type','cuisines','restaurant_type','online_order','book_table','dish_liked','cuisines']].groupby(['average_cost'], sort = True)
newdf=newdf.filter(lambda x: x['average_cost'].mean() <= 1500)
newdf=newdf.sort_values(by='average_cost')

newdf_expensive=X[['name','average_cost','locality','rest_type','cuisines','restaurant_type','online_order','book_table','dish_liked','cuisines']].groupby(['average_cost'], sort = True)
newdf_expensive=newdf_expensive.filter(lambda x: x['average_cost'].mean() >= 3000)
newdf_expensive=newdf_expensive.sort_values(by='average_cost')

In [ ]:
newdf

In [ ]:
newdf_rate=X[['name','rate']].groupby(['rate'], sort = True)
newdf_rate=newdf_rate.filter(lambda x: x['rate'].mean() >= 4.5)
newdf_rate=newdf_rate.sort_values(by=['rate'])
X.rate.value_counts()
X.rate.unique()
X.nunique()

In [ ]:
s1 = pd.merge(newdf,newdf_rate,how='inner',on=['name'])

Cheapest restrauents with low cost and high rating are....

In [ ]:
s1.head(60)

In [ ]:
s2 = pd.merge(newdf_expensive,newdf_rate,how='inner',on=['name'])

expensive restrauents with high price and high rating are.....

In [ ]:
s2

In [ ]:
newdf_voting = X[['name','average_cost','votes','locality','rest_type','cuisines','restaurant_type','online_order','book_table','dish_liked','cuisines']]

In [ ]:
newdf_voting=X[['name','average_cost','votes','locality','rest_type','cuisines','restaurant_type','online_order','book_table','dish_liked','cuisines']].groupby(['votes'], sort = True)


In [ ]:
newdf_voting=newdf_voting.filter(lambda x: x['votes'].mean() >= 200)

In [ ]:
newdf_voting=newdf_voting.sort_values(by=['votes'])

In [ ]:
s3 = pd.merge(pd.merge(newdf_voting,newdf_rate,on='name'),newdf,on='name')

Restrauent with high rating and high voting and cheap in terms of money

In [ ]:
s3.iloc[:,:12]

In [ ]:
s4 = pd.merge(pd.merge(newdf_voting,newdf_rate,on='name'),newdf_expensive,on='name')

Restrauent with high rating and high voting and expensive in terms of money

In [ ]:
s4.iloc[:,:12]